In [1]:
# import dependencies
import pandas as pd

In [2]:
# read csv file
file = "Resources\Crime_Data_from_2020_to_Present.csv"
la_crime = pd.read_csv(file, encoding="utf-8")
la_crime.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,10304468,1/8/2020 0:00,1/8/2020 0:00,2230,3,Southwest,377,2,624,BATTERY - SIMPLE ASSAULT,...,AO,Adult Other,624.0,NaN,NaN,NaN,1100 W 39TH PL,NaN,34.0141,-118.2978
1,190101086,1/2/2020 0:00,1/1/2020 0:00,330,1,Central,163,2,624,BATTERY - SIMPLE ASSAULT,...,IC,Invest Cont,624.0,NaN,NaN,NaN,700 S HILL ST,NaN,34.0459,-118.2545
2,200110444,4/14/2020 0:00,2/13/2020 0:00,1200,1,Central,155,2,845,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE,...,AA,Adult Arrest,845.0,NaN,NaN,NaN,200 E 6TH ST,NaN,34.0448,-118.2474
3,191501505,1/1/2020 0:00,1/1/2020 0:00,1730,15,N Hollywood,1543,2,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),...,IC,Invest Cont,745.0,998.0,NaN,NaN,5400 CORTEEN PL,NaN,34.1685,-118.4019
4,191921269,1/1/2020 0:00,1/1/2020 0:00,415,19,Mission,1998,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",...,IC,Invest Cont,740.0,NaN,NaN,NaN,14400 TITUS ST,NaN,34.2198,-118.4468


In [3]:
la_crime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690454 entries, 0 to 690453
Data columns (total 28 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   DR_NO           690454 non-null  int64  
 1   Date Rptd       690454 non-null  object 
 2   DATE OCC        690454 non-null  object 
 3   TIME OCC        690454 non-null  int64  
 4   AREA            690454 non-null  int64  
 5   AREA NAME       690454 non-null  object 
 6   Rpt Dist No     690454 non-null  int64  
 7   Part 1-2        690454 non-null  int64  
 8   Crm Cd          690454 non-null  int64  
 9   Crm Cd Desc     690454 non-null  object 
 10  Mocodes         595482 non-null  object 
 11  Vict Age        690454 non-null  int64  
 12  Vict Sex        599971 non-null  object 
 13  Vict Descent    599965 non-null  object 
 14  Premis Cd       690446 non-null  float64
 15  Premis Desc     690092 non-null  object 
 16  Weapon Used Cd  239966 non-null  float64
 17  Weapon Des

In [8]:
# check values of LOCATION variables
print("Area Code") # use as primary key for LA Area table, keep as forign key in Crimes table
print("-" * 30)
print(la_crime["AREA"].value_counts())
print("*" * 74)
print("LA Area Name") # use in LA Area table
print("-" * 30)
print(la_crime["AREA NAME"].value_counts())
print("*" * 74)
print("District Number") # after inspection - not very informative
print("-" * 30)
print(la_crime["Rpt Dist No"].value_counts()) 
print("*" * 74)
print("Part 1-2") # after inspection - not very informative
print("-" * 30)
print(la_crime["Part 1-2"].value_counts()) 
print("*" * 74)
print("Location") # after inspection - not very informative
print("-" * 30)
print(la_crime["LOCATION"].value_counts()) 
print("*" * 74)
print("Premise Code") # use as primary key for Premise table, keep as forign key in Crimes table
print("-" * 30)
print(la_crime["Premis Cd"].value_counts()) 
print("*" * 74)
print("Premise Description") # use in Premise table (consider recode low counts into 'other' or filter these records out)
print("-" * 30)
print(la_crime["Premis Desc"].value_counts()) 
print("*" * 74)

Area Code
------------------------------
1     46118
12    43776
14    39776
3     38762
6     37113
18    35527
20    34645
15    34458
13    34343
7     32542
2     31909
8     31839
11    30016
9     29088
10    28832
5     28770
21    27870
17    27804
19    27769
4     26095
16    23402
Name: AREA, dtype: int64
**************************************************************************
LA Area Name
------------------------------
Central        46118
77th Street    43776
Pacific        39776
Southwest      38762
Hollywood      37113
Southeast      35527
Olympic        34645
N Hollywood    34458
Newton         34343
Wilshire       32542
Rampart        31909
West LA        31839
Northeast      30016
Van Nuys       29088
West Valley    28832
Harbor         28770
Topanga        27870
Devonshire     27804
Mission        27769
Hollenbeck     26095
Foothill       23402
Name: AREA NAME, dtype: int64
**************************************************************************
District Number
-

In [5]:
# check values of VICTIM variables
print("Mocodes") # not sure what this code is; use DR_NO as the primary key for the Victim table
print("-" * 30)
print(la_crime["Mocodes"].value_counts())
print("*" * 74)
print("Victim Age") # Looks like some data points are inaccurate (e.g., is age 0 a baby or missing value?)
print("-" * 30)
print(la_crime["Vict Age"].value_counts())  
print("*" * 74)
print("Victim Gender") # Need to figure out what are Xs and what are Hs
print("-" * 30)
print(la_crime["Vict Sex"].value_counts())
print("*" * 74)
print("Victim Ethnicity") # Need to figure out all the codes
print("-" * 30)
print(la_crime["Vict Descent"].value_counts())
print("*" * 74)

Mocodes
------------------------------
344                                   28374
329                                   16007
1822 0344                              7888
1501                                   6866
0344 1822                              5647
                                      ...  
1823 0413 0515 0513 0913 1402 2034        1
0344 0329 1609 1300 1307 0400             1
0344 1300 1606 1822 1213                  1
0100 1814 0913                            1
0344 0416 0446 0429                       1
Name: Mocodes, Length: 235585, dtype: int64
**************************************************************************
Victim Age
------------------------------
 0      167295
 30      15986
 35      15566
 31      15209
 29      15202
         ...  
 97         51
 98         51
-1          36
-2           3
 120         1
Name: Vict Age, Length: 102, dtype: int64
**************************************************************************
Victim Gender
--------------------

In [9]:
# check values of CRIME TYPE variables
print("Crime Code") # use as primary key for Crime Type table, keep as forign key in Crimes table
print("-" * 30)
print(la_crime["Crm Cd"].value_counts())
print("*" * 74)
print("Crime Code Description") # use in Crime Type table (consider filtering out records that have low counts)
print("-" * 30)
print(la_crime["Crm Cd Desc"].value_counts())  
print("*" * 74)

Crime Code
------------------------------
510    74370
624    54412
354    45014
330    43293
740    42553
       ...  
452        3
884        2
349        2
445        1
882        1
Name: Crm Cd, Length: 138, dtype: int64
**************************************************************************
Crime Code Description
------------------------------
VEHICLE - STOLEN                                           74370
BATTERY - SIMPLE ASSAULT                                   54412
THEFT OF IDENTITY                                          45014
BURGLARY FROM VEHICLE                                      43293
VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)    42553
                                                           ...  
PICKPOCKET, ATTEMPT                                            3
FAILURE TO DISPERSE                                            2
GRAND THEFT / AUTO REPAIR                                      2
DISHONEST EMPLOYEE ATTEMPTED THEFT                           

In [10]:
# check values of CRIME variables
print("Weapon Used Code") 
print("-" * 30)
print(la_crime["Weapon Used Cd"].value_counts())
print("*" * 74)
print("Weapon Description") 
print("-" * 30)
print(la_crime["Weapon Desc"].value_counts())  
print("*" * 74)
print("Crime Status") # Need ro understand codes
print("-" * 30)
print(la_crime["Status"].value_counts())  
print("*" * 74)
print("Crime Code 1") 
print("-" * 30)
print(la_crime["Crm Cd 1"].value_counts())  
print("*" * 74)        

Weapon Used Code
------------------------------
400.0    128861
500.0     24693
511.0     17620
102.0     14857
109.0      5525
          ...  
120.0         4
119.0         2
123.0         1
121.0         1
124.0         1
Name: Weapon Used Cd, Length: 79, dtype: int64
**************************************************************************
Weapon Description
------------------------------
STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)    128861
UNKNOWN WEAPON/OTHER WEAPON                        24693
VERBAL THREAT                                      17620
HAND GUN                                           14857
SEMI-AUTOMATIC PISTOL                               5525
                                                   ...  
MAC-11 SEMIAUTOMATIC ASSAULT WEAPON                    4
MAC-10 SEMIAUTOMATIC ASSAULT WEAPON                    2
M1-1 SEMIAUTOMATIC ASSAULT RIFLE                       1
HECKLER & KOCH 91 SEMIAUTOMATIC ASSAULT RIFLE          1
M-14 SEMIAUTOMATIC ASSAULT RIFLE  